In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B3,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

2025-04-14 10:12:15.961952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744625536.160655      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744625536.216701      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

I0000 00:00:1744625580.966792      31 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1744625580.968903      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [5]:
x_train=tf.keras.applications.efficientnet.preprocess_input(x_train)
x_test=tf.keras.applications.efficientnet.preprocess_input(x_test)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model=EfficientNetV2B3(
    weights='imagenet',
    include_top=False, 
    input_shape=(64,64,3) 
)

52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
base_model.trainable=False

In [10]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b3 (Functional)       │ (None, 2, 2, 1536)          │      12,930,622 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1536)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,573,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,193,506 (57.96 MB)

 Trainable params: 2,259,300 (8.62 MB)

 Non-trainable params: 12,934,206 (49.34 MB)

In [11]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [12]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [13]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [14]:
import time
start_time=time.time()

In [15]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [16]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 105s 80ms/step - accuracy: 0.0495 - loss: 4.8158 - val_accuracy: 0.3469 - val_loss: 2.6505
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 69s 53ms/step - accuracy: 0.2057 - loss: 3.4026 - val_accuracy: 0.4204 - val_loss: 2.2618
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 62s 53ms/step - accuracy: 0.2629 - loss: 3.0065 - val_accuracy: 0.4530 - val_loss: 2.0785
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 61s 52ms/step - accuracy: 0.3015 - loss: 2.8092 - val_accuracy: 0.4731 - val_loss: 1.9767
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 61s 52ms/step - accuracy: 0.3164 - loss: 2.6872 - val_accuracy: 0.4870 - val_loss: 1.9059
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 63s 54ms/step - accuracy: 0.3338 - loss: 2.6123 - val_accuracy: 0.4992 - val_loss: 1.8534
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 64s 54ms/step - accuracy: 0.3520 - loss: 2.5439 - val_accuracy: 0.5057 - val_loss: 1.8133
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 64s 54ms/step - accuracy: 0.3689 

In [17]:
head_training_time=time.time()-start_time

In [18]:
base_model.trainable=True

In [19]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [20]:
start_fine_tune=time.time()

In [21]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


E0414 10:35:20.067917      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:35:20.250356      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:35:20.573692      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:35:20.763195      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:35:21.232604      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:35:21.460994      31 gpu_timer.cc:82] Del

  30/1172 ━━━━━━━━━━━━━━━━━━━━ 19:46 1s/step - accuracy: 0.1488 - loss: 4.1594

E0414 10:36:28.441994      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:36:28.625338      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:36:28.951106      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:36:29.141790      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:36:29.605922      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 10:36:29.833547      31 gpu_timer.cc:82] Del

1172/1172 ━━━━━━━━━━━━━━━━━━━━ 215s 150ms/step - accuracy: 0.1600 - loss: 3.9638 - val_accuracy: 0.3362 - val_loss: 2.6982
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 124s 82ms/step - accuracy: 0.2509 - loss: 3.1609 - val_accuracy: 0.4190 - val_loss: 2.2591
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 98s 84ms/step - accuracy: 0.3143 - loss: 2.7541 - val_accuracy: 0.4660 - val_loss: 1.9980
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 97s 83ms/step - accuracy: 0.3521 - loss: 2.5502 - val_accuracy: 0.5042 - val_loss: 1.8307
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 99s 84ms/step - accuracy: 0.3843 - loss: 2.3711 - val_accuracy: 0.5237 - val_loss: 1.7338


In [22]:
fine_tune_time=time.time()-start_fine_tune

In [23]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.3418 - loss: 2.6826

Test Accuracy: 34.05%
Total training time: 1932.56 seconds


In [24]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3389 - loss: 2.6863
Validation accuracy: 33.6160%


In [25]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step


In [26]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for EfficientNetV2B3:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for EfficientNetV2B3:
Accuracy: 0.3405
Precision: 0.3399
Recall: 0.3405
F1-Score: 0.3339


In [27]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["EfficientNetV2B3"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [28]:
print(results)

              Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  EfficientNetV2B3         0.3405              0.33616   2.674706   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1297.801288            634.755404      15193506  


In [29]:
results.to_csv("EfficientNetV2B3.csv",index=False)
print("\nResults saved to EfficientNetV2B3.csv")


Results saved to EfficientNetV2B3.csv
